In [1]:
import torchvision
import torch
from torch import nn
import torch.nn.functional as F
import torchvision.models as models
import torch.optim as optim
import copy
import os
from tqdm.autonotebook import tqdm
import matplotlib.pyplot as plt
from torch.utils.data import Dataset
from torchvision import transforms
from torch.utils.data import DataLoader
import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler
import cv2
import sys

In [2]:
from model import resnet50
model = resnet50(class_num=8)
model.load_state_dict(torch.load("c3d_29.h5"), strict=False)
model.eval()
# 저장된 model을 불러와서 확인

SlowFast(
  (fast_conv1): Conv3d(3, 8, kernel_size=(5, 7, 7), stride=(1, 2, 2), padding=(2, 3, 3), bias=False)
  (fast_bn1): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (fast_relu): ReLU(inplace=True)
  (fast_maxpool): MaxPool3d(kernel_size=(1, 3, 3), stride=(1, 2, 2), padding=(0, 1, 1), dilation=1, ceil_mode=False)
  (fast_res1): Sequential(
    (0): Bottleneck(
      (conv1): Conv3d(8, 8, kernel_size=(3, 1, 1), stride=(1, 1, 1), padding=(1, 0, 0), bias=False)
      (bn1): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv3d(8, 8, kernel_size=(1, 3, 3), stride=(1, 1, 1), padding=(0, 1, 1), bias=False)
      (bn2): BatchNorm3d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv3d(8, 32, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
      (bn3): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
     

In [3]:
import av
import glob
import os
import time
import tqdm
import datetime
import argparse

In [4]:
def extract_frames(video_path):
    frames = []
    video = av.open(video_path)
    for frame in video.decode(0):
        yield frame.to_image()

In [5]:
from tqdm.autonotebook import tqdm
path = 'content/test'
result = 'content/preprocessed_test'

for i in tqdm(os.listdir(path)):
  p1 = os.path.join(path,i)
  r1 = os.path.join(result,i)
  if os.path.exists(r1):
            continue
  os.makedirs(r1,exist_ok = True)
  for j in os.listdir(p1):
    vid_path = os.path.join(p1,j)
    r2 = os.path.join(r1,j[:-4])
    os.makedirs(r2,exist_ok = True)
    for j, frame in enumerate((extract_frames(vid_path))):
      frame.save(os.path.join(r2, f"{j}.jpg"))

In [6]:
import cv2
import numpy as np

path = 'content/preprocessed_test'
res = 'content/crime16_test'
#Number
seq_length = 16

def preprocess_data(seq_length,path,res):
  dir = os.listdir(path)
  for i in tqdm(dir):
      p1 = os.path.join(path,i)
      r1 = os.path.join(res,i)
      os.makedirs(r1,exist_ok = True)
      for j in os.listdir(p1):
          p2 = os.path.join(p1,j)
          r2 = os.path.join(r1,j)
          l = 0
          skip_length = int(len(os.listdir(p2))/seq_length)
          for m in range(1):
              k = m
              while(l!=seq_length):

                  p3 = os.path.join(p2,str(k) + ".jpg")
                  try:
                      img = cv2.imread(p3)
                      img = cv2.resize(img,(128,128))
                  except:
                      print(p3)
                  if(k==0):
                      img1 = img
                  else:
                      img1 = np.append(img1,img,axis = 1)
                  k = k+skip_length
                  l = l+1    
              cv2.imwrite(r2 + str(m)+".jpg",img1)
        

In [7]:
preprocess_data(seq_length, path, res)

In [8]:
#Label file:
classes = ['Abuse', 'Arrest', 'Arson', 'Assault', 'Burglary', 'Explosion', 'Fighting', 'normal']
decoder = {}
for i in range(len(classes)):
    decoder[classes[i]] = i
encoder = {}
for i in range(len(classes)):
    encoder[i] = classes[i]

In [9]:
test_id = list()
path = 'content/crime16_test'
for i in os.listdir(path):
  p1 = os.path.join(path,i)
  for j in os.listdir(p1):
    p2 = os.path.join(p1,j)
    test_id.append((i,p2))
print(test_id)

[('normal', 'content/crime16_test\\normal\\NV_7550.jpg')]


In [10]:
class video_dataset(Dataset):
    def __init__(self,frame_list,sequence_length = 16,transform = None):
        self.frame_list = frame_list
        self.transform = transform
        self.sequence_length = sequence_length
    def __len__(self):
        return len(self.frame_list)
    def __getitem__(self,idx):
        label,path = self.frame_list[idx]
        img = cv2.imread(path)
        seq_img = list()
        for i in range(16):
          img1 = img[:,128*i:128*(i+1),:]
          if(self.transform):
            img1 = self.transform(img1)
          seq_img.append(img1)
        seq_image = torch.stack(seq_img)
        seq_image = seq_image.reshape(3,16,im_size,im_size)
        return seq_image,decoder[label]

In [11]:
im_size = 128
mean = [0.4889, 0.4887, 0.4891]
std = [0.2074, 0.2074, 0.2074]

test_transforms = transforms.Compose([
                                        transforms.ToPILImage(),
                                        transforms.Resize((im_size,im_size)),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean,std)])
test_data = video_dataset(test_id,sequence_length = 16,transform = test_transforms)
test_loader = DataLoader(test_data,batch_size = 1,num_workers = 0 ,shuffle = True)
# data를 전처리하고 test_lodaer 객체를 생성

In [12]:
import statistics 
import os
from statistics import mode 
  
def most_common(List): 
    return(mode(List)) 

dataiter = iter(test_loader)
images, labels = dataiter.next()
images = images
outputs = model(images)
print(outputs)
_, predicted = torch.max(outputs, 1)

print('Predicted: ', ' '.join('%5s' % classes[predicted[0]]))
if torch.cuda.is_available():
    torch.cuda.empty_cache()
# 예측값 출력

tensor([[ 1.0126, -0.1103,  0.8912, -2.6302,  0.8928, -1.5646,  4.7064, -3.4317]],
       grad_fn=<AddmmBackward>)
Predicted:  F i g h t i n g


In [13]:
import sys
cctv_id = '123456789'
f = open('index.html', 'w')
sys.stdout = f
print("<!doctype html>\n<html>\n\t<head>\n\t<meta charset=\"utf-8\">\n\t<title>\n\t\tPolice Server\n\t</title>")
print("\t<style>\n\t\t*{margin:0;padding:0;}\n\t\tbody{font:20\"NanumGothic\"}")
print()
print("\t\t#d1 {\n\t\t\twidth:250px;\n\t\t\theight:60px;\n\t\t\tbackground:#ff1100;\n\t\t\tmargin:0 auto;\n\t\t}")
print("\t\th1{\n\t\t\ttext-align:center;\n\t\t\tline-height:60px;\n\t\t}")
print()
print("\t</style>\n</head>")
print("<body>")
print("\t<div id=\"d1\">\n\t\t<h1>" + classes[predicted[0]] + " Alert !")
print("CCTV ID:" + cctv_id)
print("</h1>\n\t</div>\n</body>\n</html>")
sys.stdout = sys.__stdout__
f.close()

In [14]:
if classes[predicted[0]] != 'normal':
    !scp C:\deep_cctv\index.html root@192.168.8.106:/shared

ssh: connect to host 192.168.8.106 port 22: Connection timed out
lost connection
